# Importar bibliotecas

In [2]:
import pandas as pd
import numpy as np

from scipy.stats import randint

#Pré-processamento
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_validate

#Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

#Refinamentos e métricas
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#Dados desbalanceados
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as pp_smt

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

np.set_printoptions(threshold=None, precision=2)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('precision', 3)
pd.set_option('float_format', '{:.4f}'.format)

In [8]:
df_lote_devolvido_PF = pd.read_csv('conjunto_dados_unificados_transformados.csv')
df_lote_devolvido_PF.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10075 entries, 0 to 10074
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Número                       10075 non-null  int64  
 1   NI Beneficiário              10075 non-null  int64  
 2   Origem                       10075 non-null  object 
 3   Situação                     10075 non-null  object 
 4   Valor                        10075 non-null  float64
 5   NIs_iguais                   10075 non-null  object 
 6   Tempo_decorrido              10075 non-null  int64  
 7   Natureza do Processo         10075 non-null  object 
 8   Forma de Cadastramento       10075 non-null  object 
 9   Crédito                      10075 non-null  object 
 10  Situação Cadastral - Código  10075 non-null  int64  
 11  País - Código                10075 non-null  int64  
 12  Estrangeiro                  10075 non-null  object 
 13  Nacionalidade - 

## Remoção de 'NI Beneficiário' e 'Número'

In [9]:
df_am = df_lote_devolvido_PF.drop(columns=['NI Beneficiário', 'Número'])

## Colunas categóricas

In [4]:
colunas_categoricas = ['Origem', 'NIs_iguais', 'Natureza do Processo', 'Forma de Cadastramento', 'Crédito',
                       'Situação Cadastral - Código', 'País - Código', 'Estrangeiro', 'Nacionalidade - Código',
                       'Sexo - Código', 'UF', 'Município - Código', 'Telefone - DDD', 'Situação']

for coluna in colunas_categoricas:
    df_am[coluna] = pd.Categorical(df_am[coluna], ordered=False)
df_am.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10075 entries, 0 to 10074
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Origem                       10075 non-null  category
 1   Situação                     10075 non-null  category
 2   Valor                        10075 non-null  float64 
 3   NIs_iguais                   10075 non-null  category
 4   Tempo_decorrido              10075 non-null  int64   
 5   Natureza do Processo         10075 non-null  category
 6   Forma de Cadastramento       10075 non-null  category
 7   Crédito                      10075 non-null  category
 8   Situação Cadastral - Código  10075 non-null  category
 9   País - Código                10075 non-null  category
 10  Estrangeiro                  10075 non-null  category
 11  Nacionalidade - Código       10075 non-null  category
 12  Sexo - Código                10075 non-null  category
 13  U

## X, y

In [5]:
X = df_am.drop(columns='Situação', axis=1)
le = LabelEncoder()
y = le.fit_transform(df_am['Situação'])
colunas_numericas = X.select_dtypes(exclude='category').columns.to_list()
colunas_categoricas = X.select_dtypes(include='category').columns.to_list()
print(f'Tamanhos dos conjuntos de dados X e rótulo de dados y: \n{X.shape} \n{y.shape}')
print(f'\nColunas numericas: \n{colunas_numericas} \n\nColunas categóricas: \n{colunas_categoricas}')

Tamanhos dos conjuntos de dados X e rótulo de dados y: 
(10075, 17) 
(10075,)

Colunas numericas: 
['Valor', 'Tempo_decorrido', 'Idade', 'Tempo de Óbito'] 

Colunas categóricas: 
['Origem', 'NIs_iguais', 'Natureza do Processo', 'Forma de Cadastramento', 'Crédito', 'Situação Cadastral - Código', 'País - Código', 'Estrangeiro', 'Nacionalidade - Código', 'Sexo - Código', 'UF', 'Município - Código', 'Telefone - DDD']


# Pipeline

In [6]:
def predicao(X, y, colunas_numericas, colunas_categoricas, classificador):
    transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
    transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                     ('cat', transformador_categorico, colunas_categoricas)])

    pipeline = Pipeline(steps=[('preprocessador', preprocessador), ('classificador',  classificador)])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=0)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
        
    print(f'\nAcurácia de previsão: {accuracy_score(y_test, y_pred)*100:.2f}%')
    relatorio_classificacao = classification_report(y_test, y_pred, target_names=['CREDITADA', 'DEVOLVIDA'])
    rel_class = classification_report(y_test, y_pred, target_names=['CREDITADA', 'DEVOLVIDA'], output_dict=True)
    print(f'\n******************* Relatório de classificação ********************\n\n'+relatorio_classificacao)
    
    cnf_matrix = confusion_matrix(y_test, y_pred)
    cnf_table = pd.DataFrame(data=cnf_matrix, index=['CREDITADA', 'DEVOLVIDA'], columns=['CREDITADA (prev)', 'DEVOLVIDA (prev)'])
    
    print('\n*********************** Tabela de confusão ************************\n','\n',cnf_table)
    
    #validação cruzada
    cv=StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
    resultados = cross_validate(pipeline, X, y, cv=cv)
    print(f'\nAcurácia média por validação cruzada: {resultados["test_score"].mean()*100:.2f}%')
    return cnf_table, resultados["test_score"], rel_class

# Modelos

## Árvore de Decisão 

In [7]:
classificador = DecisionTreeClassifier(random_state=0)
dtc_cm, dtc_vc, dtc_rc = predicao(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 96.43%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      0.98      0.98      2363
   DEVOLVIDA       0.72      0.70      0.71       156

    accuracy                           0.96      2519
   macro avg       0.85      0.84      0.84      2519
weighted avg       0.96      0.96      0.96      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2320                43
DEVOLVIDA                47               109

Acurácia média por validação cruzada: 96.20%


### RandomizedSearchCV

In [8]:
classificador = DecisionTreeClassifier()

transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                 ('cat', transformador_categorico, colunas_categoricas)])

clf = Pipeline(steps=[('preprocessador', preprocessador), ('model',  classificador)])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

parametros = {'model__criterion':['entropy', 'gini'],
              'model__splitter':['best', 'random'],
              'model__max_depth':randint(2, 20),
              'model__min_samples_split':randint(2, 7), 
              'model__min_samples_leaf':randint(1, 7), 
              'model__random_state':[0]}

clf = RandomizedSearchCV(clf, parametros, random_state=0, n_jobs=-1)
search = clf.fit(X_train, y_train)

print('\nOs melhores parâmetros encontrados:')
for par, value in  search.best_params_.items():
  print(par, ': ', value)


Os melhores parâmetros encontrados:
model__criterion :  gini
model__max_depth :  15
model__min_samples_leaf :  1
model__min_samples_split :  3
model__random_state :  0
model__splitter :  best


In [9]:
classificador = DecisionTreeClassifier(random_state=0, splitter=search.best_params_['model__splitter'],
                                       criterion=search.best_params_['model__criterion'],
                                       max_depth=search.best_params_['model__max_depth'],
                                       min_samples_split=search.best_params_['model__min_samples_split'],
                                       min_samples_leaf=search.best_params_['model__min_samples_leaf'])

dtc_cm_rs, dtc_vc_rs, dtc_rc_rs  = predicao(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 97.14%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.97      1.00      0.98      2363
   DEVOLVIDA       0.93      0.58      0.72       156

    accuracy                           0.97      2519
   macro avg       0.95      0.79      0.85      2519
weighted avg       0.97      0.97      0.97      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2356                 7
DEVOLVIDA                65                91

Acurácia média por validação cruzada: 96.73%


## SVC  


In [10]:
classificador = svm.SVC(random_state=0)
svc_cm, svc_vc, svc_rs = predicao(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 96.71%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.97      1.00      0.98      2363
   DEVOLVIDA       0.89      0.53      0.67       156

    accuracy                           0.97      2519
   macro avg       0.93      0.76      0.82      2519
weighted avg       0.97      0.97      0.96      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2353                10
DEVOLVIDA                73                83

Acurácia média por validação cruzada: 96.44%


### RandomizedSearchCV

In [11]:
classificador = svm.SVC()
transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                 ('cat', transformador_categorico, colunas_categoricas)])

pipeline = Pipeline(steps=[('preprocessador', preprocessador), ('model',  classificador)])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

parametros = parametros = {'model__C':randint(0, 10), 
              'model__kernel':['poly', 'linear', 'sigmoid'],
              'model__max_iter':[5000]} 

clf = RandomizedSearchCV(pipeline, parametros, n_jobs=-1, random_state=0)
search = clf.fit(X_train, y_train)

print('\nOs melhores parâmetros encontrados:')
for par, value in  search.best_params_.items():
  print(par, ': ', value)

In [37]:
classificador = svm.SVC(C=search.best_params_['model__C'],
                        kernel=search.best_params_['model__kernel'])
svc_cm_gs, svc_vc_gs, svc_rc_gs  = predicao(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 97.02%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      0.99      0.98      2363
   DEVOLVIDA       0.84      0.64      0.73       156

    accuracy                           0.97      2519
   macro avg       0.91      0.82      0.86      2519
weighted avg       0.97      0.97      0.97      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2344                19
DEVOLVIDA                56               100

Acurácia média por validação cruzada: 97.08%


## Random Forest

In [19]:
classificador = RandomForestClassifier(n_jobs=-1, random_state=0)
rf_cm, rf_vc, rf_rc = predicao(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 97.62%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      1.00      0.99      2363
   DEVOLVIDA       0.95      0.65      0.77       156

    accuracy                           0.98      2519
   macro avg       0.97      0.82      0.88      2519
weighted avg       0.98      0.98      0.97      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2358                 5
DEVOLVIDA                55               101

Acurácia média por validação cruzada: 97.38%


### RandomizedSearchCV

In [56]:
classificador = RandomForestClassifier()

transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                 ('cat', transformador_categorico, colunas_categoricas)])

pipeline = Pipeline(steps=[('preprocessador', preprocessador),('estimator',  classificador)])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

parametros = {'estimator__criterion':['gini', 'entropy'],
              'estimator__n_estimators':randint(50, 500),
              'estimator__max_depth':randint(1, 15),
              'estimator__min_samples_leaf':randint(1, 10),
              'estimator__random_state':[0]} 

clf = RandomizedSearchCV(pipeline, parametros, random_state=0, n_jobs=-1)
search = clf.fit(X_train, y_train)
print('\nOs melhores parâmetros encontrados:')
for par, value in  search.best_params_.items():
  print(par, ': ', value)


Os melhores parâmetros encontrados:
estimator__criterion :  gini
estimator__max_depth :  14
estimator__min_samples_leaf :  1
estimator__n_estimators :  455
estimator__random_state :  0


In [58]:
classificador = RandomForestClassifier(criterion=search.best_params_['estimator__criterion'],  
                                       max_depth=search.best_params_['estimator__max_depth'],  
                                       min_samples_leaf=search.best_params_['estimator__min_samples_leaf'],                                                                         
                                       n_estimators=search.best_params_['estimator__n_estimators'],
                                       random_state=0)
rf_cm_rs, rf_vc_rs, rf_rc_rs = predicao(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 96.47%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.96      1.00      0.98      2363
   DEVOLVIDA       1.00      0.43      0.60       156

    accuracy                           0.96      2519
   macro avg       0.98      0.71      0.79      2519
weighted avg       0.97      0.96      0.96      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2363                 0
DEVOLVIDA                89                67

Acurácia média por validação cruzada: 96.09%


# Tratamendo do desbalanceamento de classes

In [61]:
creditadas = df_am[df_am['Situação']=='CREDITADA'].copy()
devolvidas = df_am[df_am['Situação']=='DEVOLVIDA'].copy()
creditadas.shape, devolvidas.shape

((9450, 18), (625, 18))

## SVC - Aplicação de peso

In [23]:
# a opção 'balanced' tem como resultado:
y.shape/(2*np.bincount(y))

array([0.53, 8.06])

In [24]:
classificador = svm.SVC()
transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                 ('cat', transformador_categorico, colunas_categoricas)])

clf = Pipeline(steps=[('preprocessador', preprocessador), ('model',  classificador)])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

parametros = {'model__class_weight':[{1:1}, {1:3}, {1:5}, {1:8}, {1:10}]}

clf = GridSearchCV(clf, parametros, n_jobs=-1)
search = clf.fit(X_train, y_train)

print('\nOs melhores parâmetros encontrados:')
for par, value in  search.best_params_.items():
  print(par, ': ', value)


Os melhores parâmetros encontrados:
model__class_weight :  {1: 3}


In [25]:
classificador = svm.SVC(class_weight={1:3})
svc_cm_103 , svc_vc_103, svc_rc_103 = predicao(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 97.02%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      0.99      0.98      2363
   DEVOLVIDA       0.86      0.62      0.72       156

    accuracy                           0.97      2519
   macro avg       0.92      0.81      0.85      2519
weighted avg       0.97      0.97      0.97      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2347                16
DEVOLVIDA                59                97

Acurácia média por validação cruzada: 96.79%


## Divisão 1:1

In [26]:
creditadas = df_am[df_am['Situação']=='CREDITADA'].copy()
devolvidas = df_am[df_am['Situação']=='DEVOLVIDA'].copy()
creditadas.shape, devolvidas.shape

((9450, 18), (625, 18))

In [27]:
creditadas_sample = creditadas.sample(n=625, random_state=0)
creditadas_sample.shape

(625, 18)

In [28]:
df_balanceado = pd.concat([devolvidas, creditadas_sample], axis=0)
df_balanceado.shape

(1250, 18)

In [29]:
X_bal = df_balanceado.drop(columns='Situação', axis=1)
y_bal = le.fit_transform(df_balanceado['Situação'])

### Decision tree

In [30]:
classificador = DecisionTreeClassifier(random_state=0)
dtc_cm_11, dtc_vc_11, dtc_rc_11  = predicao(X_bal, y_bal, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 81.15%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.81      0.81      0.81       157
   DEVOLVIDA       0.81      0.81      0.81       156

    accuracy                           0.81       313
   macro avg       0.81      0.81      0.81       313
weighted avg       0.81      0.81      0.81       313


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               127                30
DEVOLVIDA                29               127

Acurácia média por validação cruzada: 81.20%


### SVC

In [31]:
classificador = svm.SVC(random_state=0)
svc_cm_11 , svc_vc_11, svc_rc_11 = predicao(X_bal, y_bal, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 80.51%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.79      0.83      0.81       157
   DEVOLVIDA       0.82      0.78      0.80       156

    accuracy                           0.81       313
   macro avg       0.81      0.81      0.80       313
weighted avg       0.81      0.81      0.81       313


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               130                27
DEVOLVIDA                34               122

Acurácia média por validação cruzada: 81.76%


### Randon Forest

In [32]:
classificador = RandomForestClassifier(n_jobs=-1, random_state=0)
rf_cm_11, rf_vc_11, rf_rc_11 = predicao(X_bal, y_bal, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 82.43%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.80      0.86      0.83       157
   DEVOLVIDA       0.85      0.79      0.82       156

    accuracy                           0.82       313
   macro avg       0.83      0.82      0.82       313
weighted avg       0.83      0.82      0.82       313


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               135                22
DEVOLVIDA                33               123

Acurácia média por validação cruzada: 85.04%


## Divisão 2:1

In [33]:
creditadas_sample_2_1 = creditadas.sample(n=1250, random_state=0)
creditadas_sample_2_1.shape

(1250, 18)

In [34]:
df_balanceado = pd.concat([devolvidas, creditadas_sample_2_1], axis=0)
df_balanceado.shape

(1875, 18)

In [35]:
X_bal2 = df_balanceado.drop(columns='Situação', axis=1)
y_bal2 = le.fit_transform(df_balanceado['Situação'])

### Decision Tree

In [36]:
classificador = DecisionTreeClassifier(random_state=0)
dtc_cm_21, dtc_vc_21, dtc_rc_21 = predicao(X_bal2, y_bal2, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 80.17%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.87      0.83      0.85       313
   DEVOLVIDA       0.68      0.75      0.72       156

    accuracy                           0.80       469
   macro avg       0.78      0.79      0.78       469
weighted avg       0.81      0.80      0.80       469


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               259                54
DEVOLVIDA                39               117

Acurácia média por validação cruzada: 85.59%


### SVC

In [37]:
classificador = svm.SVC(random_state=0)
svc_cm_21 , svc_vc_21, svc_rc_21 = predicao(X_bal2, y_bal2, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 84.86%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.85      0.95      0.89       313
   DEVOLVIDA       0.86      0.65      0.74       156

    accuracy                           0.85       469
   macro avg       0.85      0.80      0.82       469
weighted avg       0.85      0.85      0.84       469


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               296                17
DEVOLVIDA                54               102

Acurácia média por validação cruzada: 84.57%


### Random Forest

In [38]:
classificador = RandomForestClassifier(random_state=0)
rf_cm_21, rf_vc_21, rf_rc_21 = predicao(X_bal2, y_bal2, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 85.50%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.86      0.94      0.90       313
   DEVOLVIDA       0.85      0.68      0.76       156

    accuracy                           0.86       469
   macro avg       0.85      0.81      0.83       469
weighted avg       0.85      0.86      0.85       469


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               295                18
DEVOLVIDA                50               106

Acurácia média por validação cruzada: 86.86%


## Divisão 5:1

In [39]:
creditadas_sample_5_1 = creditadas.sample(n=3125, random_state=0)
creditadas_sample_5_1.shape

(3125, 18)

In [40]:
df_balanceado = pd.concat([devolvidas, creditadas_sample_5_1], axis=0)
df_balanceado.shape

(3750, 18)

In [41]:
X_bal5 = df_balanceado.drop(columns='Situação', axis=1)
y_bal5 = le.fit_transform(df_balanceado['Situação'])

### Decision Tree

In [42]:
classificador = DecisionTreeClassifier(random_state=0)
dtc_cm_51, dtc_vc_51, dtc_rc_51 = predicao(X_bal5, y_bal5, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 91.36%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.96      0.94      0.95       782
   DEVOLVIDA       0.72      0.79      0.75       156

    accuracy                           0.91       938
   macro avg       0.84      0.87      0.85       938
weighted avg       0.92      0.91      0.92       938


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               733                49
DEVOLVIDA                32               124

Acurácia média por validação cruzada: 92.45%


### SVC

In [43]:
classificador = svm.SVC(random_state=0)
svc_cm_51, svc_vc_51, svc_rc_51 = predicao(X_bal5, y_bal5, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 91.36%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.91      0.99      0.95       782
   DEVOLVIDA       0.91      0.53      0.67       156

    accuracy                           0.91       938
   macro avg       0.91      0.76      0.81       938
weighted avg       0.91      0.91      0.90       938


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               774                 8
DEVOLVIDA                73                83

Acurácia média por validação cruzada: 91.97%


### Random Forest

In [44]:
classificador = RandomForestClassifier(random_state=0)
rf_cm_51, rf_vc_51, rf_rc_51 = predicao(X_bal5, y_bal5, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 94.56%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.95      0.98      0.97       782
   DEVOLVIDA       0.90      0.76      0.82       156

    accuracy                           0.95       938
   macro avg       0.93      0.87      0.90       938
weighted avg       0.94      0.95      0.94       938


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA               769                13
DEVOLVIDA                38               118

Acurácia média por validação cruzada: 93.57%


## Divisão 10:1

In [62]:
creditadas_sample_10_1 = creditadas.sample(n=6250, random_state=0)
creditadas_sample_10_1.shape

(6250, 18)

In [63]:
df_balanceado = pd.concat([devolvidas, creditadas_sample_10_1], axis=0)
df_balanceado.shape

(6875, 18)

In [64]:
X_bal10 = df_balanceado.drop(columns='Situação', axis=1)
y_bal10 = le.fit_transform(df_balanceado['Situação'])

### Decision Tree

In [48]:
classificador = DecisionTreeClassifier(random_state=0)
dtc_cm_101, dtc_vc_101, dtc_rc_101 = predicao(X_bal10, y_bal10, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 94.36%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.96      0.98      0.97      1563
   DEVOLVIDA       0.72      0.63      0.67       156

    accuracy                           0.94      1719
   macro avg       0.84      0.80      0.82      1719
weighted avg       0.94      0.94      0.94      1719


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              1524                39
DEVOLVIDA                58                98

Acurácia média por validação cruzada: 95.00%


### SVC

In [49]:
classificador = svm.SVC(random_state=0)
svc_cm_101, svc_vc_101, svc_rc_101 = predicao(X_bal10, y_bal10, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 94.82%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.95      0.99      0.97      1563
   DEVOLVIDA       0.90      0.48      0.63       156

    accuracy                           0.95      1719
   macro avg       0.93      0.74      0.80      1719
weighted avg       0.95      0.95      0.94      1719


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              1555                 8
DEVOLVIDA                81                75

Acurácia média por validação cruzada: 95.04%


### Random Forest

In [65]:
classificador = RandomForestClassifier(random_state=0)
rf_cm_101, rf_vc_101, rf_rc_101 = predicao(X_bal10, y_bal10, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 95.70%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.96      1.00      0.98      1563
   DEVOLVIDA       0.95      0.56      0.70       156

    accuracy                           0.96      1719
   macro avg       0.95      0.78      0.84      1719
weighted avg       0.96      0.96      0.95      1719


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              1558                 5
DEVOLVIDA                69                87

Acurácia média por validação cruzada: 96.50%


In [70]:
classificador = RandomForestClassifier()

transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                 ('cat', transformador_categorico, colunas_categoricas)])

pipeline = Pipeline(steps=[('preprocessador', preprocessador),('estimator',  classificador)])

X_train, X_test, y_train, y_test = train_test_split(X_bal10, y_bal10, random_state=0, test_size=0.25)

parametros = {'estimator__criterion':['gini', 'entropy'],
              'estimator__n_estimators':randint(50, 500),
              'estimator__max_depth':randint(1, 15),
              'estimator__min_samples_leaf':randint(1, 10),
              'estimator__random_state':[0]} 

clf = RandomizedSearchCV(pipeline, parametros, random_state=0)
search = clf.fit(X_train, y_train)
print('\nOs melhores parâmetros encontrados:')
for par, value in  search.best_params_.items():
  print(par, ': ', value)


Os melhores parâmetros encontrados:
estimator__criterion :  gini
estimator__max_depth :  11
estimator__min_samples_leaf :  2
estimator__n_estimators :  89
estimator__random_state :  0


In [71]:
classificador = RandomForestClassifier(criterion=search.best_params_['estimator__criterion'],  
                                       max_depth=search.best_params_['estimator__max_depth'],  
                                       min_samples_leaf=search.best_params_['estimator__min_samples_leaf'],                                                                         
                                       n_estimators=search.best_params_['estimator__n_estimators'],
                                       random_state=0)
rf_cm_101_rs, rf_vc_101_rs, rf_rc_101_rs = predicao(X_bal10, y_bal10, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 93.08%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.93      1.00      0.96      1563
   DEVOLVIDA       1.00      0.24      0.38       156

    accuracy                           0.93      1719
   macro avg       0.96      0.62      0.67      1719
weighted avg       0.94      0.93      0.91      1719


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              1563                 0
DEVOLVIDA               119                37

Acurácia média por validação cruzada: 93.60%


## SMOTE

In [53]:
def predicao_SMOTE(X, y, colunas_numericas, colunas_categoricas, classificador):
    transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
    transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                     ('cat', transformador_categorico, colunas_categoricas)])

    pipeline = pp_smt(steps=[('preprocessador', preprocessador),
                             ('smote', SMOTE(random_state=0)),
                             ('model',  classificador)])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y, test_size=0.25)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    relatorio_classificacao = classification_report(y_test, y_pred, target_names=['CREDITADA', 'DEVOLVIDA'], zero_division=1)
    rel_class = classification_report(y_test, y_pred, target_names=['CREDITADA', 'DEVOLVIDA'], zero_division=1, output_dict=True)

    print(f'\nAcurácia de previsão: {accuracy_score(y_test, y_pred)*100:.2f}%')
    print(f'\n******************* Relatório de classificação ********************\n\n'+relatorio_classificacao)
    
    cnf_matrix = confusion_matrix(y_test, y_pred)
    cnf_table = pd.DataFrame(data=cnf_matrix, index=['CREDITADA', 'DEVOLVIDA'], columns=['CREDITADA (prev)', 'DEVOLVIDA (prev)'])
    print('\n*********************** Tabela de confusão ************************\n','\n',cnf_table)
    
    #validação cruzada
    cv=StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
    resultados = cross_validate(pipeline, X, y, cv=cv)
    print(f'\nAcurácia média por validação cruzada: {resultados["test_score"].mean()*100:.2f}%')
    return cnf_table, resultados["test_score"], rel_class

### Decision Tree

In [54]:
classificador = DecisionTreeClassifier(random_state=0)
dtc_cm_SMT, dtc_vc_SMT, dtc_rc_SMT = predicao_SMOTE(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 95.83%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      0.97      0.98      2363
   DEVOLVIDA       0.65      0.71      0.68       156

    accuracy                           0.96      2519
   macro avg       0.81      0.84      0.83      2519
weighted avg       0.96      0.96      0.96      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2303                60
DEVOLVIDA                45               111

Acurácia média por validação cruzada: 95.35%


### SVC

In [55]:
classificador = svm.SVC(random_state=0)
svc_cm_SMT , svc_vc_SMT, svc_rc_SMT = predicao_SMOTE(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 93.29%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      0.94      0.96      2363
   DEVOLVIDA       0.47      0.76      0.58       156

    accuracy                           0.93      2519
   macro avg       0.73      0.85      0.77      2519
weighted avg       0.95      0.93      0.94      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2232               131
DEVOLVIDA                38               118

Acurácia média por validação cruzada: 93.25%


### Random Forest

In [56]:
classificador = RandomForestClassifier(random_state=0)
rf_cm_SMT, rf_vc_SMT, rf_rc_SMT = predicao_SMOTE(X, y, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 97.26%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      0.99      0.99      2363
   DEVOLVIDA       0.87      0.66      0.75       156

    accuracy                           0.97      2519
   macro avg       0.92      0.83      0.87      2519
weighted avg       0.97      0.97      0.97      2519


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2347                16
DEVOLVIDA                53               103

Acurácia média por validação cruzada: 97.03%


# Tratando outlier

In [83]:
fig = px.histogram(df_am, x='Valor', marginal='box')
fig

In [84]:
percentil25 = df_am['Valor'].quantile(0.25)
percentil75 = df_am['Valor'].quantile(0.75)
iqr = percentil75 - percentil25
min, max = percentil25-1.5*iqr, percentil75+1.5*iqr
min, max

(-860.3949999999999, 1744.205)

In [85]:
df_am_outlier = df_am[(df_am['Valor']>min)&(df_am['Valor']<max)]
df_am_outlier.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8616 entries, 0 to 10073
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Origem                       8616 non-null   category
 1   Situação                     8616 non-null   category
 2   Valor                        8616 non-null   float64 
 3   NIs_iguais                   8616 non-null   category
 4   Tempo_decorrido              8616 non-null   int64   
 5   Natureza do Processo         8616 non-null   category
 6   Forma de Cadastramento       8616 non-null   category
 7   Crédito                      8616 non-null   category
 8   Situação Cadastral - Código  8616 non-null   category
 9   País - Código                8616 non-null   category
 10  Estrangeiro                  8616 non-null   category
 11  Nacionalidade - Código       8616 non-null   category
 12  Sexo - Código                8616 non-null   category
 13  UF

In [86]:
fig = px.histogram(df_am_outlier, x='Valor', marginal='box')
fig

## Quantidade de registros excluídos

In [87]:
df_am.shape[0]-df_am_outlier.shape[0]

1459

In [63]:
df_am_outlier['Valor'].describe()

count   8616.0000
mean     347.9369
std      364.7181
min        0.0100
25%      104.5850
50%      203.8900
75%      488.9800
max     1738.5500
Name: Valor, dtype: float64

## X,y sem outlier

In [64]:
X_out = df_am_outlier.drop(columns='Situação', axis=1)
le = LabelEncoder()
y_out = le.fit_transform(df_am_outlier['Situação'])
colunas_numericas = X_out.select_dtypes(exclude='category').columns.to_list()
colunas_categoricas = X_out.select_dtypes(include='category').columns.to_list()
print(f'Tamanhos dos conjuntos de dados X e rótulo de dados y: \n{X_out.shape} \n{y_out.shape}')
print(f'\nColunas numericas: \n{colunas_numericas} \n\nColunas categóricas: \n{colunas_categoricas}')

Tamanhos dos conjuntos de dados X e rótulo de dados y: 
(8616, 17) 
(8616,)

Colunas numericas: 
['Valor', 'Tempo_decorrido', 'Idade', 'Tempo de Óbito'] 

Colunas categóricas: 
['Origem', 'NIs_iguais', 'Natureza do Processo', 'Forma de Cadastramento', 'Crédito', 'Situação Cadastral - Código', 'País - Código', 'Estrangeiro', 'Nacionalidade - Código', 'Sexo - Código', 'UF', 'Município - Código', 'Telefone - DDD']


## Árvore de Decisão 

In [65]:
classificador = DecisionTreeClassifier(random_state=0)
dtc_out_cm, dtc_out_vc, dtc_out_rc = predicao(X_out, y_out, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 96.80%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      0.98      0.98      2013
   DEVOLVIDA       0.76      0.75      0.75       141

    accuracy                           0.97      2154
   macro avg       0.87      0.87      0.87      2154
weighted avg       0.97      0.97      0.97      2154


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              1979                34
DEVOLVIDA                35               106

Acurácia média por validação cruzada: 96.45%


### RandomizedSearchCV

In [66]:
classificador = DecisionTreeClassifier()

transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                 ('cat', transformador_categorico, colunas_categoricas)])

clf = Pipeline(steps=[('preprocessador', preprocessador), ('model',  classificador)])

X_train, X_test, y_train, y_test = train_test_split(X_out, y_out, random_state=0, test_size=0.25)

parametros = {'model__criterion':['entropy', 'gini'],
              'model__splitter':['best', 'random'],
              'model__max_depth':randint(2, 20),
              'model__min_samples_split':randint(2, 7), 
              'model__min_samples_leaf':randint(1, 7), 
              'model__random_state':[0]}

clf = RandomizedSearchCV(clf, parametros, random_state=0, n_jobs=-1, n_iter=10)
search = clf.fit(X_train, y_train)

print('\nOs melhores parâmetros encontrados:')
for par, value in  search.best_params_.items():
  print(par, ': ', value)


Os melhores parâmetros encontrados:
model__criterion :  entropy
model__max_depth :  15
model__min_samples_leaf :  1
model__min_samples_split :  3
model__random_state :  0
model__splitter :  random


In [67]:
classificador = DecisionTreeClassifier(random_state=0, splitter=search.best_params_['model__splitter'],
                                       criterion=search.best_params_['model__criterion'],
                                       max_depth=search.best_params_['model__max_depth'],
                                       min_samples_split=search.best_params_['model__min_samples_split'],
                                       min_samples_leaf=search.best_params_['model__min_samples_leaf'])

dtc_out_cm_rs, dtc_out_vc_rs, dtc_out_rc_rs  = predicao(X_out, y_out, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 97.21%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      1.00      0.99      2013
   DEVOLVIDA       0.90      0.65      0.75       141

    accuracy                           0.97      2154
   macro avg       0.94      0.82      0.87      2154
weighted avg       0.97      0.97      0.97      2154


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2003                10
DEVOLVIDA                50                91

Acurácia média por validação cruzada: 97.12%


## SVC  


In [68]:
classificador = svm.SVC(random_state=0)
svc_out_cm, svc_out_vc, svc_out_rc = predicao(X_out, y_out, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 96.47%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.97      0.99      0.98      2013
   DEVOLVIDA       0.87      0.55      0.67       141

    accuracy                           0.96      2154
   macro avg       0.92      0.77      0.83      2154
weighted avg       0.96      0.96      0.96      2154


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2001                12
DEVOLVIDA                64                77

Acurácia média por validação cruzada: 96.43%


### RandomizedSearchCV

In [59]:
classificador = svm.SVC()
transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                 ('cat', transformador_categorico, colunas_categoricas)])

pipeline = Pipeline(steps=[('preprocessador', preprocessador), ('model',  classificador)])

X_train, X_test, y_train, y_test = train_test_split(X_out, y_out, random_state=0, test_size=0.25)

parametros = parametros = {'model__C':randint(0, 10), 
              'model__kernel':['poly', 'linear', 'sigmoid'],
              'model__max_iter':[5000]} 

clf = RandomizedSearchCV(pipeline, parametros, n_jobs=-1, random_state=0)
search = clf.fit(X_train, y_train)

print('\nOs melhores parâmetros encontrados:')
for par, value in  search.best_params_.items():
  print(par, ': ', value)

NameError: name 'X_out' is not defined

In [70]:
classificador = svm.SVC(C=search.best_params_['model__C'],
                        kernel=search.best_params_['model__kernel'])
svc_out_cm_gs, svc_out_vc_gs, svc_out_rc_gs  = predicao(X_out, y_out, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 96.84%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      0.99      0.98      2013
   DEVOLVIDA       0.82      0.66      0.73       141

    accuracy                           0.97      2154
   macro avg       0.90      0.82      0.86      2154
weighted avg       0.97      0.97      0.97      2154


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              1993                20
DEVOLVIDA                48                93

Acurácia média por validação cruzada: 97.01%


## Random Forest

In [71]:
classificador = RandomForestClassifier(n_jobs=-1, random_state=0)
rf_out_cm, rf_out_vc, rf_out_rc = predicao(X_out, y_out, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 97.73%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.98      1.00      0.99      2013
   DEVOLVIDA       0.93      0.71      0.80       141

    accuracy                           0.98      2154
   macro avg       0.95      0.85      0.90      2154
weighted avg       0.98      0.98      0.98      2154


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2005                 8
DEVOLVIDA                41               100

Acurácia média por validação cruzada: 97.32%


### RandomizedSearchCV

In [72]:
classificador = RandomForestClassifier()

transformador_numerico = Pipeline(steps=[('scaler', StandardScaler())])
transformador_categorico = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[('num', transformador_numerico, colunas_numericas),
                                                 ('cat', transformador_categorico, colunas_categoricas)])

pipeline = Pipeline(steps=[('preprocessador', preprocessador),('estimator',  classificador)])

X_train, X_test, y_train, y_test = train_test_split(X_out, y_out, random_state=0, test_size=0.25)

parametros = {'estimator__criterion':['gini', 'entropy'],
              'estimator__n_estimators':randint(50, 500),
              'estimator__max_depth':randint(1, 15),
              'estimator__min_samples_leaf':randint(1, 10),
              'estimator__random_state':[0]} 

clf = RandomizedSearchCV(pipeline, parametros, random_state=0, n_jobs=-1, n_iter=10)
search = clf.fit(X_train, y_train)
print('\nOs melhores parâmetros encontrados:')
for par, value in  search.best_params_.items():
  print(par, ': ', value)


Os melhores parâmetros encontrados:
estimator__criterion :  gini
estimator__max_depth :  11
estimator__min_samples_leaf :  2
estimator__n_estimators :  89
estimator__random_state :  0


In [73]:
classificador = RandomForestClassifier(criterion=search.best_params_['estimator__criterion'],  
                                       max_depth=search.best_params_['estimator__max_depth'],  
                                       min_samples_leaf=search.best_params_['estimator__min_samples_leaf'],                                                                         
                                       n_estimators=search.best_params_['estimator__n_estimators'],
                                       random_state=0)
rf_out_cm_rs, rf_out_vc_rs, rf_out_rc_rs = predicao(X_out, y_out, colunas_numericas, colunas_categoricas, classificador)


Acurácia de previsão: 95.45%

******************* Relatório de classificação ********************

              precision    recall  f1-score   support

   CREDITADA       0.95      1.00      0.98      2013
   DEVOLVIDA       0.98      0.31      0.47       141

    accuracy                           0.95      2154
   macro avg       0.97      0.66      0.72      2154
weighted avg       0.96      0.95      0.94      2154


*********************** Tabela de confusão ************************
 
            CREDITADA (prev)  DEVOLVIDA (prev)
CREDITADA              2012                 1
DEVOLVIDA                97                44

Acurácia média por validação cruzada: 95.15%


# Resultados

## Acurácia na validação cruzada

In [109]:
data = {'DTC':dtc_vc, 'DTC_rs':dtc_vc_rs,'SVC':svc_vc,'SVC_rs':svc_vc_gs,'RF':rf_vc,
        'RF_rs':rf_vc_rs, 'DTC_11':dtc_vc_11, 'DTC_21':dtc_vc_21, 'DTC_51':dtc_vc_51,
        'DTC_101':dtc_vc_101, 'DTC_SMT':dtc_vc_SMT, 'SVC_11':svc_vc_11,
        'SVC_21':svc_vc_21, 'SVC_51':svc_vc_51, 'SVC_103':svc_vc_103, 'SVC_101':svc_vc_101,
        'SVC_SMT':svc_vc_SMT, 'RF_11':rf_vc_11, 'RF_21':rf_vc_21, 'RF_51':rf_vc_51,
        'RF_101':rf_vc_101, 'RF_SMT':rf_vc_SMT, 'DTC_out': dtc_out_vc, 'SVC_out':svc_out_vc, 'RF_out':rf_out_vc,
        'DTC_out_rs': dtc_out_vc_rs, 'SVC_out_gs':svc_out_vc_gs, 'RF_out_rs':rf_out_vc_rs}
acuracias = pd.DataFrame.from_dict(data, orient='index')
acuracias['média'] = acuracias.mean(axis=1)
acuracias.sort_values('média')

,0,1,2,3,4,5,6,7,8,9,média
DTC_11,0.8320,0.8160,0.8720,0.8560,0.8080,0.7840,0.7280,0.7600,0.8400,0.8240,0.8120
SVC_11,0.8560,0.7920,0.8080,0.8480,0.8400,0.8320,0.7760,0.7840,0.8400,0.8000,0.8176
SVC_21,0.8138,0.8511,0.8617,0.7926,0.8245,0.8989,0.8617,0.8723,0.8191,0.8617,0.8457
RF_11,0.8800,0.8560,0.8800,0.8480,0.8560,0.8400,0.8080,0.8000,0.8800,0.8560,0.8504
DTC_21,0.8191,0.8298,0.8936,0.8511,0.8351,0.8830,0.8830,0.8298,0.8989,0.8351,0.8559
RF_21,0.8511,0.8723,0.9096,0.8085,0.8670,0.9096,0.8830,0.8564,0.8883,0.8404,0.8686
SVC_51,0.9227,0.9227,0.9200,0.9333,0.9120,0.9360,0.9147,0.9067,0.9040,0.9253,0.9197
DTC_51,0.9333,0.9253,0.9173,0.9440,0.9040,0.9280,0.9387,0.9040,0.9173,0.9333,0.9245
SVC_SMT,0.9167,0.9236,0.9474,0.9345,0.9335,0.9296,0.9296,0.9464,0.9296,0.9345,0.9325
RF_51,0.9573,0.9387,0.9333,0.9493,0.9147,0.9333,0.9493,0.9307,0.9227,0.9280,0.9357


## Relatórios de classificação

In [113]:
relatorios = [dtc_rc, dtc_rc_rs, svc_rs, svc_rc_gs, rf_rc, rf_rc_rs, svc_rc_103, dtc_rc_11, svc_rc_11,
              rf_rc_11, dtc_rc_21, svc_rc_21, rf_rc_21, dtc_rc_51, svc_rc_51, rf_rc_51, dtc_rc_101,
              svc_rc_101, rf_rc_101, dtc_rc_SMT, svc_rc_SMT, rf_rc_SMT, dtc_out_rc, svc_out_rc, rf_out_rc,
              dtc_out_rc_rs, svc_out_rc_gs, rf_out_rc_rs]
nomes = ['dtc', 'dtc_rs', 'svc', 'svc_gs', 'rfC', 'rfC_rs', 'svc_103', 'dtc_11', 'svc_11',
              'rfC_11', 'dtc_21', 'svc_21', 'rfC_21', 'dtc_51', 'svc_51', 'rfC_51', 'dtc_101',
              'svc_101', 'rfC_101', 'dtc_SMT', 'svc_SMT', 'rfC_SMT', 'DTC_out', 'SVC_out', 'RFC_out',
              'DTC_out_rs', 'SVC_out_Gs', 'RFC_out_rs']

rel_class = {}

for r,n in zip(relatorios, nomes):
  rel_class[str.upper(n)]=r

df_rc = pd.DataFrame()

for k,v in rel_class.items():
  df_rci = pd.DataFrame(v)
  df_rci['modelo'] = k
  df_rci['metricas'] = df_rci.index
  df_rci.set_index('modelo', inplace=True)
  df_rc = df_rc.append(df_rci)

df_rc

,CREDITADA,DEVOLVIDA,accuracy,macro avg,weighted avg,metricas
modelo,,,,,,
DTC,0.9801,0.7171,0.9643,0.8486,0.9639,precision
DTC,0.9818,0.6987,0.9643,0.8403,0.9643,recall
DTC,0.9810,0.7078,0.9643,0.8444,0.9641,f1-score
DTC,2363.0000,156.0000,0.9643,2519.0000,2519.0000,support
DTC_RS,0.9743,0.8704,0.9698,0.9223,0.9678,precision
DTC_RS,0.9941,0.6026,0.9698,0.7983,0.9698,recall
DTC_RS,0.9841,0.7121,0.9698,0.8481,0.9672,f1-score
DTC_RS,2363.0000,156.0000,0.9698,2519.0000,2519.0000,support
SVC_RS,0.9699,0.8925,0.9671,0.9312,0.9651,precision


## Tabelas de confusão

In [116]:
matrizes_confusao = [dtc_cm, dtc_cm_rs, svc_cm, svc_cm_gs, rf_cm, rf_cm_rs, svc_cm_103 , dtc_cm_11, 
                     svc_cm_11, rf_cm_11, dtc_cm_21, svc_cm_21, rf_cm_21, dtc_cm_51, svc_cm_51,
                     rf_cm_51, dtc_cm_101, svc_cm_101, rf_cm_101,dtc_cm_SMT, svc_cm_SMT, rf_cm_SMT,
                     dtc_out_cm, svc_out_cm, rf_out_cm, dtc_out_cm_rs, svc_out_cm_gs, rf_out_cm_rs]

nomes = ['dtc_cm', 'dtc_cm_rs', 'svc_cm', 'svc_cm_gs', 'rf_cm', 'rf_cm_rs', 'svc_cm_103' , 'dtc_cm_11', 
         'svc_cm_11', 'rf_cm_11', 'dtc_cm_21', 'svc_cm_21', 'rf_cm_21', 'dtc_cm_51', 'svc_cm_51',
         'rf_cm_51', 'dtc_cm_101', 'svc_cm_101', 'rf_cm_101','dtc_cm_SMT', 'svc_cm_SMT', 'rf_cm_SMT',
         'DTC_out', 'SVC_out', 'RF_out', 'DTC_out_rs', 'SVC_out_rs', 'RF_out']

dict_matrizes = {}
for m,n in zip(matrizes_confusao, nomes):
  dict_matrizes[str.upper(n)] = m

df_relatorios = pd.DataFrame()
for k,v in dict_matrizes.items():
  df = dict_matrizes[k]
  df['Totais'] = df['CREDITADA (prev)'] + df['DEVOLVIDA (prev)']
  creditadas = df.iloc[[0],[2]].astype(int).to_numpy()[0][0]
  devolvidas = df.iloc[[1],[2]].astype(int).to_numpy()[0][0]
  df1=pd.DataFrame.from_dict(v).reset_index().melt(id_vars=['index'], value_vars=['CREDITADA (prev)',	'DEVOLVIDA (prev)'])
  df1['Modelo'] = k
  df1['Tot. Cred.'] = creditadas
  df1['Tot. Dev.'] = devolvidas
  df_relatorios = df_relatorios.append(df1, ignore_index=True)
  
df_relatorios

,index,variable,value,Modelo,Tot. Cred.,Tot. Dev.
0,CREDITADA,CREDITADA (prev),2320,DTC_CM,2363,156
1,DEVOLVIDA,CREDITADA (prev),47,DTC_CM,2363,156
2,CREDITADA,DEVOLVIDA (prev),43,DTC_CM,2363,156
3,DEVOLVIDA,DEVOLVIDA (prev),109,DTC_CM,2363,156
4,CREDITADA,CREDITADA (prev),2349,DTC_CM_RS,2363,156
5,DEVOLVIDA,CREDITADA (prev),62,DTC_CM_RS,2363,156
6,CREDITADA,DEVOLVIDA (prev),14,DTC_CM_RS,2363,156
7,DEVOLVIDA,DEVOLVIDA (prev),94,DTC_CM_RS,2363,156
8,CREDITADA,CREDITADA (prev),2353,SVC_CM,2363,156
9,DEVOLVIDA,CREDITADA (prev),73,SVC_CM,2363,156
